# Save & Load single cells and trajectoty data from json file

In [1]:
# import some common libraries
import os
import glob
import os.path
import numpy as np
import json, random, cv2
from cellpose import models
from cellpose.io import imread
import matplotlib.pyplot as plt

from tqdm import tqdm
from pathlib import Path
from skimage import measure
from PIL import Image, ImageSequence

# from livecellx import segment
from livecellx import core
from livecellx.core import datasets
from livecellx.core import SingleCellTrajectory, SingleCellStatic
from livecellx.core.datasets import LiveCellImageDataset, SingleImageDataset
from livecellx.sample_data import tutorial_three_image_sys

## SingleCellStatic: saving & loading 

In [2]:
io_out_dir = Path("test_io_output")

In [3]:
dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")


In [4]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")

time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

3 png img file paths loaded;


In [7]:
from skimage.measure import regionprops
from livecellx.core.io_sc import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:07<00:00,  2.39s/it]


In [8]:
single_cells[1].meta

{'label_in_mask': 2}

In [9]:
sc_json_list = SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=True)

In [10]:
SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=False)

In [11]:
img_dataset = single_cells[0].img_dataset

In [12]:
loaded_scs = SingleCellStatic.load_single_cells_json(io_out_dir/"single_cells.json")

In [13]:
loaded_scs[0]

SingleCellStatic(id=f51ff63c-f331-4833-bb4e-020f2e32d920, timeframe=1, bbox=[ 143.  981.  207. 1047.])

In [14]:
for sc in single_cells[:2]:
    print(sc.id)

f51ff63c-f331-4833-bb4e-020f2e32d920
b9b23429-e33a-485f-a681-aaaa2b37a91c


In [15]:
for sc in loaded_scs[:2]:
    print(sc.id)

f51ff63c-f331-4833-bb4e-020f2e32d920
b9b23429-e33a-485f-a681-aaaa2b37a91c


In [16]:
for sc in single_cells:
    for loaded_sc in loaded_scs:
        if sc.id == loaded_sc.id:
            # compare contour, contours are np.array
            assert np.allclose(sc.contour, loaded_sc.contour), f"the difference between sc.contour and loaded_sc.contour is {sc.contour - loaded_sc.contour}, ids are {sc.id} and {loaded_sc.id}"

## SingleCellTrajectory: saving & loading

In [17]:
from typing import List
from livecellx.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


sct_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

In [18]:
sct_collection.write_json(io_out_dir/"sct_collection.json")